In [1]:
import pandas as pd
import re
from multiprocessing import Pool
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from itertools import combinations

# Identificación de pares

In [2]:
metadata = pd.read_csv('/home/ferambriz/Programs/ncov_600_mil/data/custom.metadata.tsv', sep = '\t')
dict_sp = {
    'Human': 'Human',
    'Humano': 'Human'
}
metadata['host'] = metadata['host'].replace(dict_sp)
metadata = metadata[metadata['host']=='Human']

temporal_clusters = ['2020-03-01', '2020-09-01', '2021-01-01', '2021-05-01', '2022-01-01', '2022-06-01', '2022-11-01', '2023-01-01', '2023-07-01', '2023-09-06']
temporal_clusters = pd.to_datetime(temporal_clusters)


metadata['date'] = pd.to_datetime(metadata['date'])

def assign_cluster(date, clusters):
    for i in range(len(clusters)-1):
        if date >= clusters[i] and date < clusters[i+1]:
            return i
    return len(clusters)-1
    
metadata['cluster'] = metadata['date'].apply(lambda date: assign_cluster(date, temporal_clusters))


metadata.head()

,strain,virus,gisaid_epi_isl,genbank_accession,date,region,country,division,location,region_exposure,...,GISAID_clade,originating_lab,submitting_lab,authors,url,title,paper_url,date_submitted,purpose_of_sequencing,cluster
12867,hCoV-19/HongKong/VM20002508-2/2020,betacoronavirus,EPI_ISL_414567,?,2020-02-10,Asia,Hong Kong,NaN,NaN,Asia,...,O,Hong Kong Department of Health,"School of Public Health, The University of Hon...","Dominic N.C. Tsang, Daniel K.W. Chu, Leo L.M. ...",https://www.gisaid.org/,?,?,2020-03-13,?,9
12868,hCoV-19/USA/WA-UW20/2020,NaN,EPI_ISL_414368,?,2020-03-05,North America,USA,Washington,NaN,North America,...,S,UW Virology Lab,UW Virology Lab,"Pavitra Roychoudhury, Hong Xie, Keith Jerome, ...",https://www.gisaid.org/,?,?,2020-03-11,?,0
12869,hCoV-19/Canada/BC_02421/2020,betacoronavirus,EPI_ISL_415581,?,2020-03-01,North America,Canada,British Columbia,NaN,North America,...,O,BCCDC Public Health Laboratory,BCCDC Public Health Laboratory,"Harrigan, Prystajecky, Krajden, Lee, Kamelian,...",https://www.gisaid.org/,?,?,2020-03-18,?,0
12870,hCoV-19/Netherlands/NA_10/2020,betacoronavirus,EPI_ISL_415466,?,2020-03-09,Europe,Netherlands,NaN,NaN,Europe,...,G,Dutch COVID-19 response team,Erasmus Medical Center,"David Nieuwenhuijse, Bas Oude Munnink, Reina S...",https://www.gisaid.org/,?,?,2020-03-17,?,0
12871,hCoV-19/Netherlands/Rotterdam_1364740/2020,betacoronavirus,EPI_ISL_413584,?,2020-03-03,Europe,Netherlands,Zuid-Holland,Rotterdam,Europe,...,GR,unknown,Erasmus Medical Center,"David Nieuwenhuijse, Bas Oude Munnink, Reina S...",https://www.gisaid.org/,?,?,2020-03-07,?,0


In [ ]:
df_db = pd.DataFrame()
for i in tqdm(metadata['country'].unique(), desc='Progreso por país'):
    for j in metadata['cluster'].unique():
        metadata_i = metadata[(metadata['country'] == i) & (metadata['cluster'] == j)]
        lst_i = list(metadata_i['strain'])
        combinaciones = list(combinations(lst_i, 2))
        
        df_i = pd.DataFrame(combinaciones, columns=['Elemento1', 'Elemento2'])
        df_i['country'] = i; df_i['cluster'] = j
        df_i.to_csv('../Data/bin_combinaciones/bin_'+ str(i) + '_' + str(j) + '.tsv', sep = '\t')

Progreso por país:   4%|▍         | 8/189 [12:39<3:48:39, 75.80s/it]  

# Identificación de porcentaje de indentidad

In [5]:
from Bio import SeqIO
df = pd.read_csv('../Data/bin_combinaciones/bin_Canada_0.tsv', sep = '\t')
df

,Unnamed: 0,Elemento1,Elemento2,country,cluster
0,0,hCoV-19/Canada/BC_02421/2020,hCoV-19/Canada/BC_78548/2020,Canada,0
1,1,hCoV-19/Canada/BC_02421/2020,hCoV-19/Canada/BC_83163/2020,Canada,0
2,2,hCoV-19/Canada/BC_02421/2020,hCoV-19/Canada/BC_66353/2020,Canada,0
3,3,hCoV-19/Canada/BC_02421/2020,hCoV-19/Canada/BC_64686/2020,Canada,0
4,4,hCoV-19/Canada/BC_02421/2020,hCoV-19/Canada/BC_65034/2020,Canada,0
...,...,...,...,...,...
11776,11776,hCoV-19/Canada/ON_QGLO-041/2020,hCoV-19/Canada/SK-NML-388/2020,Canada,0
11777,11777,hCoV-19/Canada/ON_QGLO-041/2020,hCoV-19/Canada/SK-NML-388/2020,Canada,0
11778,11778,hCoV-19/Canada/ON_QGLO-042/2020,hCoV-19/Canada/SK-NML-388/2020,Canada,0
11779,11779,hCoV-19/Canada/ON_QGLO-042/2020,hCoV-19/Canada/SK-NML-388/2020,Canada,0


In [11]:
def extract_indenty(e1, e2):
    with open('../Data/BloqueHuman/ID_Human.fasta', 'r') as fasta_file:
        for seq_record in SeqIO.parse(fasta_file, 'fasta'):
            if seq_record.id == e1:
                seq_a = seq_record.seq
            elif seq_record.id == e2:
                seq_b = seq_record.seq
    matches = sum(res1 == res2 for res1, res2 in zip(seq_a, seq_b))
    identity_percentage = (matches / len(seq_a)) * 100
    return identity_percentage
extract_indenty('hCoV-19/Canada/BC_02421/2020', 'hCoV-19/Canada/BC_78548/2020')

98.85837082801693

In [ ]:
df['Identity'] = list(map(extract_indenty, df['Elemento1'], df['Elemento2']))
df